All parameters are seperated by a "&" character

### Place Search Request:  
url: https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters

parameters wew ill be using:
- radius = 16200 (meters, approx 10 miles) 
- keyword 
- pagetoken  (gained from next_page_token)
- key  (api_key)
- location (lat, lng formatted as: "lat%2Clng")


### Find Specific Place: 
url: https://maps.googleapis.com/maps/api/place/findplacefromtext/json?parameters"

parameters: 
- textinput : textquery or phonenumber
- input:
    - text: nearly exact name of business
    - phonenumber: international format  +12345678901
returns one place_id of most likely result



### Place Details Request:  
url: https://maps.googleapis.com/maps/api/place/details/json?parameters

parameters we will be using:
- place_id


No more than 60 results will be returned for each query. mutiple locations will have to be used for this search.

GPS Locations to use 
- 44.273518, -123.271058  :  Just North of Junction City
- 43.911540, -123.129609  :  Just South of Eugene 
- 44.135766, -123.345221  :  West of Eugene
- 44.046996, -123.102148  :  Eugene
- 45.669287, -121.538651  :  Near Hood River
- 44.823046, -123.043092  :  South of Salem
- 45.089875, -123.085669  :  North of Salem
- 45.110889, -123.317063  :  NW of Salem
- 44.986040, -123.286169  :  West of Salem
- 45.269130, -123.209947  :  N of McMinville
- 45.296668, -123.052705  :  Dundee
- 45.468841, -123.163255  :  Forest Grove
- 45.428859, -122.921556  :  Farmington
- 45.724679, -119.215730  :  Echo
- 45.916807, -118.421968  :  Milton-Freewater
- 44.817970, -117.647432  :  Baker City
- 44.376467, -121.260561  :  Madras
- 44.114786, -121.369051  :  Bend
- 43.214300, -123.467444  :  West Roseburg
- 43.355756, -123.424872  :  North Roseburg
- 43.468983, -123.240851  :  East Roseburg
- 43.631220, -123.523062  :  Kellogg
- 43.775670, -123.039664  :  Walden

In [146]:
import pandas as pd
import numpy as np
import os
import requests
import config
from time import sleep
from collections import Counter

nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
details_url = "https://maps.googleapis.com/maps/api/place/details/json?"
find_place_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

In [4]:
vineyard_list_csv = "oregon_tail_list.csv"
vineyard_list_html = os.path.join("..", "..", "..", "Complete_Oregon_Wineries_List_(Updated_2022)_Oregon_Tails_files", "sheet.html")

In [5]:
html_df = pd.read_html(vineyard_list_html, header=1)[0].drop("1", axis=1)
html_df


,Name,Hours,Street,Address,Phone
0,2Hawk Vineyard & Winery,"(Daily, 1 - 7 pm)",2335 N Phoenix Road,"Medford, OR 97504",541-779-9463
1,12 Ranch Wines,(By Appointment),4550 Burgdorf Road,"Bonanza, OR 97623",541-545-1204
2,12th & Maple Wine Company,(A custom crush facility for several wineries),1242 SE Maple Street,"Dundee, OR 97115",503-538-7724
3,15 Mile Winery,"(Sat & Sun, 11 am - 5 pm)",306 Court Street,"The Dalles, OR 97058",541-965-1445
4,51 Weeks Winemaking- At Southeast Wine Collect...,(Call for seasonal hours),2425 SE 35th Place,"Portland, OR 97214",503-706-9339
...,...,...,...,...,...
602,Zenas Wines at Séjourné,(By appointment & for special events),7700 NE Cooper Lane,"Yamhill, OR 97111",971-231-5128
603,Zerba Cellars - Walla Walla Valley - Oregon,(Mon - Sat: noon - 5 pm; Sun: noon - 4 pm),85530 Hwy. 11,"Milton-Freewater, OR 97862",541-938-9463
604,Zerba Cellars,"(Daily, 11 am - 6 pm)",810 Highway 99 W,"Dundee, OR 97115",503-537-9463
605,Zimri Cellars,-,4213 NE Zimri Drive,"Newberg, OR 97132",503-260-7529


In [7]:
html_df.to_csv(vineyard_list_csv, index=False)

In [6]:
## locations for the nearby search to gain more vineyards that are not on the list
areas_to_search = ["44.273518,-123.271058", # Just North of Junction City
"43.911540,-123.129609", # Just South of Eugene 
"44.135766,-123.345221", # West of Eugene
"44.231045,-123.311644", # West of Junction City
"44.385577,-123.330183", # Notrh of Junction City                   
"45.669287,-121.538651", # Near Hood River
"44.823046,-123.043092", # South of Salem
"45.089875,-123.085669", # North of Salem
"45.110889,-123.317063", # NW of Salem
"44.986040,-123.286169", # West of Salem
"45.228954,-123.207274", # McMinville
"45.265211,-123.028746", # Dundee
"45.346827,-123.171912", # Yamhill
"45.375533,-123.007460", # N of Newberg
"45.324865,-122.869444", # South of Newberg
"45.468841,-123.163255", # Forest Grove
"45.428859,-122.921556", # Farmington
"45.724679,-119.215730", # Echo
"45.953885, -118.451395", # West Milton-Freewater
"45.900631, -118.334322", # South Milton-Freewater
"45.947440, -118.304110", # East Milton-Freewater
"46.006372, -118.342562", # Nort Milton-Freewater
"44.817970,-117.647432", # Baker City
"44.376467,-121.260561", # Madras
"44.114786,-121.369051", # Bend
"43.214300,-123.467444", # West Roseburg
"43.355756,-123.424872", # North Roseburg
"43.468983,-123.240851", # East Roseburg
"43.631220,-123.523062", # Kellogg
"43.775670,-123.039664", # Walden
"45.632858,-121.511857"] # Hood River  

In [9]:
# payload is a dictionary that holds the parameters and their values

payload = {"keyword":"Vineyards", 
           'radius':"20000", "key":config.gkey,
           "location":"45.632858,-121.511857", 
           "inputtype":"textquery",
           "input":"Wy'East Vineyards",
           "locationbias":"250000@44.071476,-121.318079"}



In [10]:
requests.get(find_place_url+f"key={payload['key']}&input={payload['input']}&inputtype={payload['inputtype']}").json()

{'candidates': [{'place_id': 'ChIJG1BslP0FllQRMRc0KTybvYM'}], 'status': 'OK'}

In [12]:
test_save = set()
for name in html_df["Name"][:100]:
    payload["inputtype"] = "textquery"
    payload["input"] = name
    response = requests.get(find_place_url+f"key={payload['key']}&input={payload['input']}&inputtype={payload['inputtype']}").json()
    if len(response['candidates']) < 1:
        continue
    elif len(response['candidates']) > 1:
        for place in response['candidates']:
            test_save.add(place['place_id'])
    else:
        test_save.add(response['candidates'][0]['place_id'])
    
print(f"saved: {len(test_save)}", f"    ||    {(len(test_save)/100)*100}% names had unique results with the test run of 100 names." )        

saved: 93     ||    93.0% names had unique results with the test run of 100 names.


In [13]:
response = requests.get(nearby_url+ f"key={payload['key']}&location={payload['location']}&radius={payload['radius']}&keyword{payload['keyword']}").json()
response

{'html_attributions': [],
 'next_page_token': 'AcYSjRgYFPlHwPLXmluMZSDOb9G01OEohPgLoyhf8UTWGSzuaVs9p6o40rOHhooBNtnoS7nXG3wJdI0eBQKSiAxtZC_Wu-cUKJz8bC1PewhOXGYtLgJ-9AeBes0rB6ImSKh81fOpmhMrudvQ_BySitmDDCbQUEHZ8LefnDbXLwf9NKVp74GnvJl1o7q80PwQbLgTIpa6T9bg8QkWXfqXfMOdXUvwwJmFHURHkfk4qTWGX9YNyRRgQ7Q4Mx0rUmNiHXfiEC7eS7iaIEvN1ScMrIXnRXP0H7xnVHZ-MB3ZdnZzkLclr-5bIBmsCkKLDaPkB-lrL03nJ45bYwxYEwkenD-Kpa2JlziZtkyPAaZU0LRKN-XZ-iGqDRYeJ0Y_3hBGSalRdYPh--5y4lPZx9E0ZnxxBqjV5FZtPDufv7Gk_WkBSiYA',
 'results': [{'geometry': {'location': {'lat': 45.7067985, 'lng': -121.528087},
    'viewport': {'northeast': {'lat': 45.72651365044828,
      'lng': -121.4953934804756},
     'southwest': {'lat': 45.69298361668825, 'lng': -121.5650871829651}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Hood River',
   'photos': [{'height'

In [14]:
response.keys()

dict_keys(['html_attributions', 'next_page_token', 'results', 'status'])

In [15]:
response["results"]

[{'geometry': {'location': {'lat': 45.7067985, 'lng': -121.528087},
   'viewport': {'northeast': {'lat': 45.72651365044828,
     'lng': -121.4953934804756},
    'southwest': {'lat': 45.69298361668825, 'lng': -121.5650871829651}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
  'name': 'Hood River',
  'photos': [{'height': 2048,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113370112691467643248">Darel</a>'],
    'photo_reference': 'AcYSjRjwv_OvOeprn9tA0Q0GLwv0q6u8OObyEaI_y3YnEFBAVfMr6Ua4Q3jefWCZ8vVkuhIvc--ZjHiJAWqBIr00bK2hzwJKQiFkTDk2NdtRWsHCS_B1z7ziqASxgmTHVddPHqDjLicld68OGvrvTqJKT_qCtfccXjezhLT_IXA5ivHOPajl',
    'width': 3072}],
  'place_id': 'ChIJ5w8cTGEOllQRXJh9TX7P9lY',
  'reference': 'ChIJ5w8cTGEOllQRXJh9TX7P9lY',
  'scope': 'GOOGLE',
  'types': ['locality', 'political'],
  'vici

In [16]:
# save the place_id from each result to use for the place detail query
[place["place_id"] for place in response["results"]]
            

['ChIJ5w8cTGEOllQRXJh9TX7P9lY',
 'ChIJBQHqmIIOllQRHd_KPATDRmY',
 'ChIJVwsudn0OllQR3tpv0uYjyPw',
 'ChIJz99pRWkOllQRYwIB5EXrkKw',
 'ChIJdwiWj4IOllQR0ODp0RsuTbE',
 'ChIJNUPfM2kOllQRP7oGG3Z-H-0',
 'ChIJoXYHA3wOllQR00MXzkdMfng',
 'ChIJo2LcXxQOllQRlfOIChY26hE',
 'ChIJ9QRkWmMOllQRd9IXeZL3sMU',
 'ChIJr1iCHm8OllQRSsJ86Exp9CE',
 'ChIJCXRMCG8OllQRyYeCYQGTZXI',
 'ChIJ1780smYOllQRetndhC0GNj4',
 'ChIJWexw1vUOllQRHc6QPR29dbE',
 'ChIJAbgZQsIOllQRu5OztB3_ri0',
 'ChIJKT-plJQPllQRpCLurjSuHoI',
 'ChIJEdjSISAGllQRsL7kpkbEqAM',
 'ChIJo0OIC2EOllQRmdjMIlMTAcc',
 'ChIJ1dfjXBQOllQRQYsswcYW3S8',
 'ChIJqW0lGkMPllQRmTLyjQ_PPxc',
 'ChIJG6E__SEGllQRRDx1zxL1Z2Q']

In [17]:
requests.get(nearby_url+f"key={payload['key']}&pagetoken={response['next_page_token']}").json()

{'html_attributions': [],
 'next_page_token': 'AcYSjRhHSaqJOI-uYFQeO3Dd1Nm9rsCunzbfDajy9l4YnwqQOyiu4h3mkUPsSLhzB8O18jqvzpOQHG155cecH02nYrBNgQzeDbiay9PTngL4RX54TMRYvYrHIlr0vlpRIhara1TKQFTa4PTP5hO8MSnpL8t_04KXERAnCb3YPy82HvgYcCdR245IV6shdoJJuII5uKamSWGZMnrjwUR9wvBxUtrguyqJx_Ske4GjC5Hb7S7x8qoqpUbiTwcNOA5mDUW3MKeiKA1hXpvLzQ8oBV_d-RiXPd2z-7mEgCkW9Au1o0rsv07ADKOkpl2i25N71aK7YvRsudnuWc1U26ixyOTbTbhpcxgMkQKv16GSKuf_Xjw5AUlHF8Xphby6Ch7Ss4FtckFrckWmgPiPB4QE4ZVbxVcAFN-QRScHf6p39DeULbbkO1wKdf4-w6hmK1qkIAW7XF_Q1ONpvQGKXx3VoGnxG7EE9352rlnNFp9Hdvlot5jVoT-aOE2YJlAYpbR-KdELKI5cwnfLu0bO0kKUsiYFvWIZoodpsHAZSFvPEcqODjXoZY-CB1dA01vhiVIEmhjLAdQKV4MeSLMXIqTEBTDfJ1xQytx22iswdp_1u0Es8CNjm7dE8jeGU-Qs41asE4BBeg',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 45.6988326, 'lng': -121.5240476},
    'viewport': {'northeast': {'lat': 45.7002776302915,
      'lng': -121.5227060197085},
     'southwest': {'lat': 45.6975796697085, 'lng': -121.5254039802915}}},
   'icon': 'https://ma

In [19]:
## Looping Through all result pages of one search grabing the next_page_tokens and place_id's

## Global Variables requied for the loops
saved_place_ids = set()
page_token_list = []

more_results = True


while more_results:    
    if len(page_token_list) == 0:
        url = nearby_url+f"key={payload['key']}&location={payload['location']}&radius={payload['radius']}&keyword={payload['keyword']}"
        response = requests.get(url).json() 
    else:
        url = nearby_url+ f"key={payload['key']}&pagetoken={page_token_list[-1]}"
        response = requests.get(url).json() 

    for place in response["results"]:
        saved_place_ids.add(place["place_id"])
    try:
        page_token_list.append(response["next_page_token"])
    except:
        more_results = False 

## giving the api time to activate the next page token and prevent an invalid response
    sleep(2)
    

print("Number of page_tokens saved:", len(page_token_list))
print("Number of unique place_ids saved:", len(saved_place_ids))
print("==============\n===============\n===============")
    

Number of page_tokens saved: 2
Number of unique place_ids saved: 51


In [20]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids

## Global Variables requied for the loops
saved_place_ids = set()
payload['20000']

for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            url = nearby_url+f"key={payload['key']}&location={payload['location']}&radius={payload['radius']}&keyword={payload['keyword']}"
            response = requests.get(url).json() 
        else:
            url = nearby_url+ f"key={payload['key']}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")

## radius:20000 = 433 results    

0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 26
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 56
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 56
3. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 56
4. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 61
5. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 111
6. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 151
7. End for this location,

In [22]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids
# Change the radius to attempt more results.
## Global Variables requied for the loops
saved_place_ids = set()
payload["radius"] = "50000"

for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            url = nearby_url+f"key={payload['key']}&location={payload['location']}&radius={payload['radius']}&keyword={payload['keyword']}"
            response = requests.get(url).json() 
        else:
            url = nearby_url+ f"key={payload['key']}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")

## radius:50000 = 376 results

0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 58
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 74
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 74
3. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 74
4. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 86
5. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 146
6. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 201
7. End for this location,

In [23]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids
# Change the radius to attempt more results.

## Global Variables requied for the loops
saved_place_ids = set()
payload["radius"] = "30000"


for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            url = nearby_url+f"key={payload['key']}&location={payload['location']}&radius={payload['radius']}&keyword={payload['keyword']}"
            response = requests.get(url).json() 
        else:
            url = nearby_url+ f"key={payload['key']}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")
## radius:30000 = 372 results

0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 40
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 57
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 59
3. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 59
4. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 67
5. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 127
6. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 185
7. End for this location,

## Status update
- With our gps locations, a radius of 20km, 30km, and 50km were tested. 20km has the highest number of unique reuslts.
- Time to start exploring the detailed data pull from google.


In [24]:
# detail_response = requests.get(details_url+f"place_id={saved_place_ids[0]}")
first_place = saved_place_ids.pop()
first_place

'ChIJb21eb2cVolQRY6JGmq_XmwA'

In [25]:
detail_response = requests.get(details_url+f"place_id={first_place}&key={payload['key']}").json()
detail_response

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '820',
    'short_name': '820',
    'types': ['street_number']},
   {'long_name': 'Mill Creek Road',
    'short_name': 'Mill Creek Rd',
    'types': ['route']},
   {'long_name': 'Walla Walla',
    'short_name': 'Walla Walla',
    'types': ['locality', 'political']},
   {'long_name': 'Walla Walla County',
    'short_name': 'Walla Walla County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Washington',
    'short_name': 'WA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '99362', 'short_name': '99362', 'types': ['postal_code']},
   {'long_name': '8415',
    'short_name': '8415',
    'types': ['postal_code_suffix']}],
  'adr_address': '<span class="street-address">820 Mill Creek Rd</span>, <span class="locality">Walla Walla</span>, <span class="region">WA<

In [26]:
detail_response.keys()

dict_keys(['html_attributions', 'result', 'status'])

In [27]:
detail_response["result"].keys()

dict_keys(['address_components', 'adr_address', 'business_status', 'current_opening_hours', 'formatted_address', 'formatted_phone_number', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'international_phone_number', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'reviews', 'types', 'url', 'user_ratings_total', 'utc_offset', 'vicinity', 'website'])

In [28]:
detail_response["result"]

{'address_components': [{'long_name': '820',
   'short_name': '820',
   'types': ['street_number']},
  {'long_name': 'Mill Creek Road',
   'short_name': 'Mill Creek Rd',
   'types': ['route']},
  {'long_name': 'Walla Walla',
   'short_name': 'Walla Walla',
   'types': ['locality', 'political']},
  {'long_name': 'Walla Walla County',
   'short_name': 'Walla Walla County',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Washington',
   'short_name': 'WA',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United States',
   'short_name': 'US',
   'types': ['country', 'political']},
  {'long_name': '99362', 'short_name': '99362', 'types': ['postal_code']},
  {'long_name': '8415',
   'short_name': '8415',
   'types': ['postal_code_suffix']}],
 'adr_address': '<span class="street-address">820 Mill Creek Rd</span>, <span class="locality">Walla Walla</span>, <span class="region">WA</span> <span class="postal-code">99362-8415</span>, <span c

In [29]:
detail_list=[first_place]  ## trash varibale to test the api calling multiple times 
for x in range(3):
    detail_list.append(saved_place_ids.pop())  ## takes the id out of saved_place_id's and puts it in detail list

In [30]:
for place in detail_list:
    print(requests.get(details_url+f"key={payload['key']}&place_id={place}").json()["result"]["name"])
    saved_place_ids.add(place) ## puts the id back into the saved_place_id's

K Vintners
Hillside Vineyard
Cória Estates
HV Cellars


In [43]:
detail_dict= dict()
reviews_dict = dict()
geometry_dict = dict()
photos_list = list()

for i, place in enumerate(detail_list):
    if i>2:
        break
##  make the search request for the details of the place
    response = requests.get(details_url+f"key={payload['key']}&place_id={place}").json()["result"]
    detail_dict[place] = dict()
    
# ## Save all the information    
    for key in response.keys():
        if key != 'geometry':
            if key != "reviews":
                if key != 'photos':
                    detail_dict[place][key] = response[key]
                else:                    
                     for photo in response[key]:
                        photos_dict = {'photo_reference' : photo['photo_reference'],
                                       'place_id':place, 'height' : photo['height'],
                                       'width' : photo['width'], 'uploader' : photo['html_attributions'] }
                        photos_list.append(photos_dict)
                
            else:
                for review in response[key]:
                    reviews_dict[place] = dict()
                    for key2 in review:
                        reviews_dict[place][key2] = review[key2]
        else: 
            geometry_dict[place] = {
                    ## response[key]['location'][lat/lng]
                    'center_lat': response[key]['location']['lat'],
                    'center_lon': response[key]['location']['lng'],
                    ## response[key]['viewport'][northeast/southeast][lat/lng]
                    'northeast_lat': response[key]['viewport']['northeast']['lat'],
                    'northeast_lon': response[key]['viewport']['northeast']['lng'],
                    'southwest_lat': response[key]['viewport']['southwest']['lat'],
                    'southwest_lon': response[key]['viewport']['southwest']['lng']}


In [44]:
pd.DataFrame(detail_dict).transpose()

,address_components,adr_address,business_status,current_opening_hours,formatted_address,formatted_phone_number,icon,icon_background_color,icon_mask_base_uri,international_phone_number,...,rating,reference,types,url,user_ratings_total,utc_offset,vicinity,website,curbside_pickup,delivery
ChIJb21eb2cVolQRY6JGmq_XmwA,"[{'long_name': '820', 'short_name': '820', 'ty...","<span class=""street-address"">820 Mill Creek Rd...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","820 Mill Creek Rd, Walla Walla, WA 99362, USA",(509) 526-5230,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 509-526-5230,...,4.4,ChIJb21eb2cVolQRY6JGmq_XmwA,"[tourist_attraction, food, point_of_interest, ...",https://maps.google.com/?cid=43865770597720675,13,-420,"820 Mill Creek Road, Walla Walla",https://houseofsmith.com/,NaN,NaN
ChIJEbcmsP31vVQRON7nMSb-B7c,"[{'long_name': '2018', 'short_name': '2018', '...","<span class=""street-address"">2018 Steele Rd</s...",OPERATIONAL,NaN,"2018 Steele Rd, The Dalles, OR 97058, USA",(541) 296-1357,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 541-296-1357,...,NaN,ChIJEbcmsP31vVQRON7nMSb-B7c,"[general_contractor, point_of_interest, establ...",https://maps.google.com/?cid=13188789473777278520,NaN,-420,"2018 Steele Road, The Dalles",NaN,NaN,NaN
ChIJm-BEGj4GwFQRQz0MEeIFSLM,"[{'long_name': '8252', 'short_name': '8252', '...","<span class=""street-address"">8252 Redstone Ave...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","8252 Redstone Ave SE, Salem, OR 97306, USA",(503) 363-0525,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 503-363-0525,...,4.7,ChIJm-BEGj4GwFQRQz0MEeIFSLM,"[tourist_attraction, food, point_of_interest, ...",https://maps.google.com/?cid=12918581999619030339,80,-420,"8252 Redstone Avenue Southeast, Salem",http://www.coriaestates.com/,True,False


In [45]:
pd.DataFrame(reviews_dict).transpose()

,author_name,author_url,language,original_language,profile_photo_url,rating,relative_time_description,text,time,translated
ChIJb21eb2cVolQRY6JGmq_XmwA,Smiledoc,https://www.google.com/maps/contrib/1063067683...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu_Gr...,5,4 years ago,"great wine, great people, great location and f...",1522099905,False
ChIJm-BEGj4GwFQRQz0MEeIFSLM,Kay Ross,https://www.google.com/maps/contrib/1123279401...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9H6...,5,a year ago,First time at Coria and not disappointed!\nCai...,1628467472,False


In [46]:
pd.DataFrame(geometry_dict).transpose()

,center_lat,center_lon,northeast_lat,northeast_lon,southwest_lat,southwest_lon
ChIJb21eb2cVolQRY6JGmq_XmwA,46.085476,-118.250395,46.087413,-118.249234,46.084715,-118.251932
ChIJEbcmsP31vVQRON7nMSb-B7c,45.575228,-121.155773,45.576016,-121.153987,45.573318,-121.156685
ChIJm-BEGj4GwFQRQz0MEeIFSLM,44.833859,-123.047117,44.835195,-123.045148,44.832497,-123.053022


In [51]:
pd.DataFrame(photos_list)

,photo_reference,place_id,height,width,uploader
0,AcYSjRjtciJ2vL291g_nifTt5z3M5tGy2Z-UFYyZm3A-b9...,ChIJb21eb2cVolQRY6JGmq_XmwA,950,1800,"[<a href=""https://maps.google.com/maps/contrib..."
1,AcYSjRgQB5wKKfrknN3desn613VV6ldhIPOCj5hVUKuepn...,ChIJb21eb2cVolQRY6JGmq_XmwA,2448,3264,"[<a href=""https://maps.google.com/maps/contrib..."
2,AcYSjRi5Xlyx2mi79ZXVOj9M04TYEQwao3gYerlPlHWPH4...,ChIJb21eb2cVolQRY6JGmq_XmwA,2988,5312,"[<a href=""https://maps.google.com/maps/contrib..."
3,AcYSjRhhjaqVsAyZ4maghq5cfDqMz26wI5QMSxSM2RPXFA...,ChIJb21eb2cVolQRY6JGmq_XmwA,2400,1920,"[<a href=""https://maps.google.com/maps/contrib..."
4,AcYSjRi5qCxhzM2hz6OPF5jhpwUcSyteD7WNGyVAkiNT23...,ChIJb21eb2cVolQRY6JGmq_XmwA,2988,5312,"[<a href=""https://maps.google.com/maps/contrib..."
5,AcYSjRidjaFx3wBuOJ2icikvuCG0QnaMV5nJ7-uHCHtwU8...,ChIJb21eb2cVolQRY6JGmq_XmwA,2448,3264,"[<a href=""https://maps.google.com/maps/contrib..."
6,AcYSjRiJhslKgaJB28ANwKaNGmw5JEHLI-RagwFcnLo4SY...,ChIJb21eb2cVolQRY6JGmq_XmwA,2432,4320,"[<a href=""https://maps.google.com/maps/contrib..."
7,AcYSjRhh8GfLqUza3xyBpwL-hs2ktcU-oa0nnp9ppDvwbF...,ChIJb21eb2cVolQRY6JGmq_XmwA,4320,2432,"[<a href=""https://maps.google.com/maps/contrib..."
8,AcYSjRi1guqqDKPUB6bZPOG2tvE2KRxIbU80QSpd93VbEL...,ChIJb21eb2cVolQRY6JGmq_XmwA,2304,4096,"[<a href=""https://maps.google.com/maps/contrib..."
9,AcYSjRjo6_lD5S-qlIG10-VhJytj1EoDeb0m0WtSxd9bFT...,ChIJb21eb2cVolQRY6JGmq_XmwA,2448,3264,"[<a href=""https://maps.google.com/maps/contrib..."


### Ready to pull the detailed information and reviews about each vinyard in the search results

In [76]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids

## Global Variables requied for the loops
saved_place_ids = set()
payload["radius"]= "20000"

for i, name in enumerate(html_df["Name"]):
    if i%50==0:
        print(f"location {i}: {len(saved_place_ids)} saved place id's so far.  ")
        sleep(1)
    payload["inputtype"] = "textquery"
    payload["input"] = name
    url = find_place_url+f"key={payload['key']}&input={payload['input']}&inputtype={payload['inputtype']}&locationbias={payload['locationbias']}"
    response = requests.get(url).json()
    if len(response['candidates']) < 1:
        continue
    elif len(response['candidates']) > 1:
        for place in response['candidates']:
            saved_place_ids.add(place['place_id'])
    else:
        saved_place_ids.add(response['candidates'][0]['place_id'])

for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            url = nearby_url+f"key={payload['key']}&location={payload['location']}&radius={payload['radius']}&keyword={payload['keyword']}"
            response = requests.get(url).json() 
        else:
            url = nearby_url+ f"key={payload['key']}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")

## radius:20000 => Oregon Tails list + 433 Nearby search results => 751  total unique results   

location 0: 0 saved place id's so far.  
location 50: 47 saved place id's so far.  
location 100: 96 saved place id's so far.  
location 150: 144 saved place id's so far.  
location 200: 191 saved place id's so far.  
location 250: 239 saved place id's so far.  
location 300: 288 saved place id's so far.  
location 350: 334 saved place id's so far.  
location 400: 377 saved place id's so far.  
location 450: 416 saved place id's so far.  
location 500: 455 saved place id's so far.  
location 550: 499 saved place id's so far.  
location 600: 535 saved place id's so far.  
0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 0
Number of unique place_ids saved: 548
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 558
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 560
3.

In [77]:
len(saved_place_ids)

751

In [84]:

## the different tables to be saved from each query
detail_dict= dict()
reviews_dict = dict()
geometry_dict = dict()
photos_list = list()

for i, place in enumerate(saved_place_ids):
##  make the search request for the details of the place
    response = requests.get(details_url+f"key={payload['key']}&place_id={place}").json()["result"]
    detail_dict[place] = dict()
    
# ## Save all the information    
    for key in response.keys():
        if key != 'geometry':
            if key != "reviews":
                if key != 'photos':
                    detail_dict[place][key] = response[key]
                else:                    
                     for photo in response[key]:
                        photos_dict = {'photo_reference' : photo['photo_reference'],
                                       'place_id':place, 'height' : photo['height'],
                                       'width' : photo['width'], 'uploader' : photo['html_attributions'] }
                        photos_list.append(photos_dict)
                
            else:
                for review in response[key]:
                    reviews_dict[place] = dict()
                    for key2 in review:
                        reviews_dict[place][key2] = review[key2]
        else: 
            geometry_dict[place] = {
                    ## response[key]['location'][lat/lng]
                    'center_lat': response[key]['location']['lat'],
                    'center_lon': response[key]['location']['lng'],
                    ## response[key]['viewport'][northeast/southeast][lat/lng]
                    'northeast_lat': response[key]['viewport']['northeast']['lat'],
                    'northeast_lon': response[key]['viewport']['northeast']['lng'],
                    'southwest_lat': response[key]['viewport']['southwest']['lat'],
                    'southwest_lon': response[key]['viewport']['southwest']['lng']}


In [85]:
details_df = pd.DataFrame(detail_dict).transpose().reset_index()
cols = details_df.columns.to_list()
details_df = details_df[[key for key in cols if key != "index"]]
details_df

,address_components,adr_address,business_status,current_opening_hours,formatted_address,formatted_phone_number,icon,icon_background_color,icon_mask_base_uri,international_phone_number,...,vicinity,website,permanently_closed,curbside_pickup,delivery,takeout,dine_in,editorial_summary,price_level,secondary_opening_hours
0,"[{'long_name': '820', 'short_name': '820', 'ty...","<span class=""street-address"">820 Mill Creek Rd...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","820 Mill Creek Rd, Walla Walla, WA 99362, USA",(509) 526-5230,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 509-526-5230,...,"820 Mill Creek Road, Walla Walla",https://houseofsmith.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'long_name': '2444', 'short_name': '2444', '...","<span class=""street-address"">2444 Cobleigh Rd<...",CLOSED_PERMANENTLY,NaN,"2444 Cobleigh Rd, Eagle Point, OR 97524, USA",(541) 865-3648,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 541-865-3648,...,"2444 Cobleigh Road, Eagle Point",NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'long_name': '8252', 'short_name': '8252', '...","<span class=""street-address"">8252 Redstone Ave...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","8252 Redstone Ave SE, Salem, OR 97306, USA",(503) 363-0525,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 503-363-0525,...,"8252 Redstone Avenue Southeast, Salem",http://www.coriaestates.com/,NaN,True,False,NaN,NaN,NaN,NaN,NaN
3,"[{'long_name': '245', 'short_name': '245', 'ty...","<span class=""street-address"">245 N Front St</s...",CLOSED_PERMANENTLY,NaN,"245 N Front St, Central Point, OR 97502, USA",(541) 664-2218,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 541-664-2218,...,"245 North Front Street, Central Point",http://www.ledgerdavid.com/,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'long_name': '140', 'short_name': '140', 'ty...","<span class=""street-address"">140 NE Hill St</s...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","140 NE Hill St, Albany, OR 97321, USA",(541) 928-2106,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 541-928-2106,...,"140 Northeast Hill Street, Albany",http://www.vivacityspirits.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,"[{'long_name': '1211', 'short_name': '1211', '...","<span class=""street-address"">1211 N Loring St<...",OPERATIONAL,NaN,"1211 N Loring St, Portland, OR 97227, USA",(503) 567-2221,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 503-567-2221,...,"1211 North Loring Street, Portland",http://www.reverendnatshardcider.com/,NaN,NaN,True,NaN,False,"{'language': 'en', 'overview': 'A selection of...",2,"[{'open_now': True, 'periods': [{'close': {'da..."
747,"[{'long_name': '272', 'short_name': '272', 'ty...","<span class=""street-address"">272 Capital Ln</s...",CLOSED_TEMPORARILY,NaN,"272 Capital Ln, Roseburg, OR 97471, USA",(541) 784-7275,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 541-784-7275,...,"272 Capital Lane, Roseburg",http://ferrarofamilyvineyards.com/,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
748,"[{'long_name': '9605', 'short_name': '9605', '...","<span class=""street-address"">9605 NE Fox Farm ...",OPERATIONAL,"{'open_now': True, 'periods': [{'close': {'dat...","9605 NE Fox Farm Rd, Dundee, OR 97115, USA",(800) 245-2950,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 800-245-2950,...,"9605 Northeast Fox Farm Road, Dundee",https://www.thefourgraces.com/,NaN,NaN,False,NaN,NaN,NaN,NaN,"[{'open_n

In [86]:
reviews_df = pd.DataFrame(reviews_dict).transpose().reset_index()
reviews_df["place_id"] = reviews_df["index"]
reviews_df = reviews_df.drop("index", axis=1)
reviews_df

,author_name,author_url,language,original_language,profile_photo_url,rating,relative_time_description,text,time,translated,place_id
0,Smiledoc,https://www.google.com/maps/contrib/1063067683...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu_Gr...,5,4 years ago,"great wine, great people, great location and f...",1522099905,False,ChIJb21eb2cVolQRY6JGmq_XmwA
1,Kay Ross,https://www.google.com/maps/contrib/1123279401...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9H6...,5,a year ago,First time at Coria and not disappointed!\nCai...,1628467472,False,ChIJm-BEGj4GwFQRQz0MEeIFSLM
2,Joshua Smith,https://www.google.com/maps/contrib/1134752247...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu8AT...,5,2 years ago,Excellent service and solid wines. Small garde...,1599336320,False,ChIJ4zJjNBN8z1QRy_DnGt1EB-c
3,Linda Teppler (S0rceress0),https://www.google.com/maps/contrib/1049795854...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9Cs...,5,3 years ago,Just tried their Treos Brandy at the Corvallis...,1561834276,False,ChIJ6QmAe3sVwFQR7KVJoo1dw3k
4,Matias Sanchez,https://www.google.com/maps/contrib/1034946324...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9_K...,5,4 years ago,The wine here blows my mind away!,1533678636,False,ChIJPyUzT_0WlVQRPplhbRhoBv8
...,...,...,...,...,...,...,...,...,...,...,...
655,Jack Wright from Woodland,https://www.google.com/maps/contrib/1071083555...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu82Q...,5,3 years ago,Reverend Nats has a great selection of ciders ...,1557102727,False,ChIJQ_Tzy02nlVQRWU9gD3wMdjU
656,Julie Helgason,https://www.google.com/maps/contrib/1159198501...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9fe...,5,3 years ago,Best Pinot Noirs in Douglas County! Also the P...,1569618105,False,ChIJX0B_1t4ZxFQRsVefqiKvc9I
657,Alaina Danae,https://www.google.com/maps/contrib/1045108330...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu8JQ...,5,3 years ago,This winery is absolutely amazing! The staff i...,1543869171,False,ChIJsfC-ZK9BlVQRdC0UjH76x58
658,Steve Yates,https://www.google.com/maps/contrib/1147946163...,NaN,NaN,https://lh3.googleusercontent.com/a/ALm5wu3BVu...,5,3 years ago,,1546546791,False,ChIJQaTQ6gxBlVQRGMITAAYqZ6U


In [102]:
photos_df = pd.DataFrame(photos_list)
photos_df

,photo_reference,place_id,height,width,uploader
0,AcYSjRgtGNvOJbBCDQspPEYaWZsggQFQyoUTPOs7ByDCrg...,ChIJb21eb2cVolQRY6JGmq_XmwA,950,1800,"[<a href=""https://maps.google.com/maps/contrib..."
1,AcYSjRij8q2agx7LPQowGmXvZkz0yHnOqA0wlDT9XxDcIF...,ChIJb21eb2cVolQRY6JGmq_XmwA,2448,3264,"[<a href=""https://maps.google.com/maps/contrib..."
2,AcYSjRjZPeu-7PMl_xZRluZbU760vCAnhsu8-WclyotBjT...,ChIJb21eb2cVolQRY6JGmq_XmwA,2988,5312,"[<a href=""https://maps.google.com/maps/contrib..."
3,AcYSjRiu8Fhw7HQRscN-k13JSySkBAkN80ayBriOE2d0wn...,ChIJb21eb2cVolQRY6JGmq_XmwA,2400,1920,"[<a href=""https://maps.google.com/maps/contrib..."
4,AcYSjRivLwy8DdhwWr8zteFv1eJSL3rIPbVifku6TZaJAF...,ChIJb21eb2cVolQRY6JGmq_XmwA,2988,5312,"[<a href=""https://maps.google.com/maps/contrib..."
...,...,...,...,...,...
5649,AcYSjRjiP83ZQK2B_rCItb_gT8MnWg9eLz96obbthoInHG...,ChIJsfC-ZK9BlVQRdC0UjH76x58,3024,4032,"[<a href=""https://maps.google.com/maps/contrib..."
5650,AcYSjRjkHmOeTViw-oxy9VuAomcyn4XZQ7Hq3nRs5qNKTj...,ChIJsfC-ZK9BlVQRdC0UjH76x58,4000,2250,"[<a href=""https://maps.google.com/maps/contrib..."
5651,AcYSjRg69d50NGWrwKOS05Gy-03idWwoYp6nJzbEFM3-Hc...,ChIJQaTQ6gxBlVQRGMITAAYqZ6U,3456,4608,"[<a href=""https://maps.google.com/maps/contrib..."
5652,AcYSjRiQHEHezDG-8nhXPKkjPJ8DWRT2kwHePf520JtElo...,ChIJQaTQ6gxBlVQRGMITAAYqZ6U,3456,4608,"[<a href=""https://maps.google.com/maps/contrib..."


In [88]:
geometry_df = pd.DataFrame(geometry_dict).transpose().reset_index()
geometry_df["place_id"] = geometry_df["index"]
geometry_df = geometry_df.drop("index", axis=1)
geometry_df

,center_lat,center_lon,northeast_lat,northeast_lon,southwest_lat,southwest_lon,place_id
0,46.085476,-118.250395,46.087413,-118.249234,46.084715,-118.251932,ChIJb21eb2cVolQRY6JGmq_XmwA
1,42.586531,-122.602158,42.587590,-122.601288,42.584892,-122.604768,ChIJZdNgbQb3xVQRba7OWrKWhno
2,44.833859,-123.047117,44.835195,-123.045148,44.832497,-123.053022,ChIJm-BEGj4GwFQRQz0MEeIFSLM
3,42.375939,-122.920096,42.377316,-122.918691,42.374618,-122.921389,ChIJ4zJjNBN8z1QRy_DnGt1EB-c
4,44.639261,-123.094003,44.640652,-123.092491,44.637954,-123.095188,ChIJ6QmAe3sVwFQR7KVJoo1dw3k
...,...,...,...,...,...,...,...
746,45.540631,-122.680243,45.541865,-122.679020,45.539167,-122.681718,ChIJQ_Tzy02nlVQRWU9gD3wMdjU
747,43.277898,-123.445733,43.279268,-123.444236,43.276571,-123.446934,ChIJX0B_1t4ZxFQRsVefqiKvc9I
748,45.285784,-123.004797,45.287135,-123.003267,45.284437,-123.005965,ChIJsfC-ZK9BlVQRdC0UjH76x58
749,45.272650,-123.013884,45.273902,-123.012259,45.271205,-123.014957,ChIJQaTQ6gxBlVQRGMITAAYqZ6U


In [92]:
for name in enumerate(details_df["name"]):
    print(name)

(0, 'K Vintners')
(1, 'LaBrasseur Vineyard')
(2, 'Cória Estates')
(3, 'Ledger David Cellars Tasting Room')
(4, 'Vivacity Spirits LLC')
(5, 'Lumos Wine Co.')
(6, 'Gresser Vineyard')
(7, 'Fly High Vineyard')
(8, 'Pike Road Wines')
(9, 'Clear Creek Distillery')
(10, 'HV Cellars')
(11, 'Treveri Cellars')
(12, 'Richwine Vineyards')
(13, 'J.K. Carriere - Wine Tastings By Appointment')
(14, 'TeSoAria Vineyard & Winery')
(15, 'Umpqua Valley Wineries')
(16, 'Boordy Vineyards Inc')
(17, 'Wannamaker Estate Vineyard')
(18, 'Three Rivers Winery')
(19, 'Crater Lake Cellars')
(20, 'Springhouse Cellar Winery')
(21, 'Armstrong Family Winery - Downtown Walla Walla Tasting Room')
(22, 'Los Rocosos Vineyards-The Rocks District of Milton Freewater')
(23, 'Syncline Winery')
(24, 'Legacy Estate Vineyard')
(25, 'Reininger Winery')
(26, "Miller's Vineyard")
(27, 'Southwind Vineyard')
(28, 'Stardance Vineyards')
(29, 'McBeth Vineyards')
(30, 'Southwest Griffin Drive')
(31, 'Hazelfern')
(32, 'Circle K')
(33, 'Ow

In [95]:
## Based on the business names alone, some need cleaned or investigated.
##
details_df.loc[[13,146,259, 324,363,378,388,401,435,437,471,477,492,506,507,515,534,536,545,571,606,634,660,697,706,726,746], ['name', 'website']]

,name,website
13,J.K. Carriere - Wine Tastings By Appointment,https://jkcarriere.com/contact/
146,Heymann Whinery (Call For Appointment outside ...,http://www.heymannwhinery.com/
259,"Naked Wines LLC, Mailing Address",http://www.nakedwinery.com/
324,Cronos Consulting LLC,NaN
363,Northeast Griffin Oaks Street,NaN
378,Wooden Shoe Tulip Festival,http://www.woodenshoe.com/
388,Coopers Hall,https://www.coopershall.com/
401,Willamette Valley Vineyards - Carlton Wine Cor...,NaN
435,The Pines Apartments,NaN
437,The Horse Radish,http://www.thehorseradish.com/


In [118]:
## remove 259,324,363,378,435,477,506,606,634,660,697,706,746
## from the data because they do not produce wine
remove_place_ids = {id for id in details_df.loc[[259,324,363,378,435,477,506,606,634,660,697,706,746], "place_id"]}
photos_df = photos_df.drop([i for i, id in enumerate(photos_df["place_id"]) if id in remove_place_ids], axis=0)
reviews_df = reviews_df.drop([i for i, id in enumerate(reviews_df["place_id"]) if id in remove_place_ids], axis=0)
details_df = details_df.drop([i for i, id in enumerate(details_df["place_id"]) if id in remove_place_ids], axis=0)
geometry_df = geometry_df.drop([i for i, id in enumerate(geometry_df["place_id"]) if id in remove_place_ids], axis=0)
geometry_df

,center_lat,center_lon,northeast_lat,northeast_lon,southwest_lat,southwest_lon,place_id
0,46.085476,-118.250395,46.087413,-118.249234,46.084715,-118.251932,ChIJb21eb2cVolQRY6JGmq_XmwA
1,42.586531,-122.602158,42.587590,-122.601288,42.584892,-122.604768,ChIJZdNgbQb3xVQRba7OWrKWhno
2,44.833859,-123.047117,44.835195,-123.045148,44.832497,-123.053022,ChIJm-BEGj4GwFQRQz0MEeIFSLM
3,42.375939,-122.920096,42.377316,-122.918691,42.374618,-122.921389,ChIJ4zJjNBN8z1QRy_DnGt1EB-c
4,44.639261,-123.094003,44.640652,-123.092491,44.637954,-123.095188,ChIJ6QmAe3sVwFQR7KVJoo1dw3k
...,...,...,...,...,...,...,...
745,43.847431,-123.250155,43.848787,-123.248608,43.846089,-123.251306,ChIJNdln3D05wVQRP5h-7LTPjaI
747,43.277898,-123.445733,43.279268,-123.444236,43.276571,-123.446934,ChIJX0B_1t4ZxFQRsVefqiKvc9I
748,45.285784,-123.004797,45.287135,-123.003267,45.284437,-123.005965,ChIJsfC-ZK9BlVQRdC0UjH76x58
749,45.272650,-123.013884,45.273902,-123.012259,45.271205,-123.014957,ChIJQaTQ6gxBlVQRGMITAAYqZ6U


In [150]:
## Oregon sw point 42.007926, -124.341437
## Oregon NW point 46.504562, -116.759725
## geometry row 750 is somewhere in europe. lets keep the data to the PNW for now
## lat goal : 40 < x < 50   ||    lon goal : -126 < x < -110
bad_locations = {geometry_df['place_id'][i] for i in geometry_df.index if not 40<geometry_df['southwest_lat'][i]<50}
[bad_locations.add(geometry_df['place_id'][i]) for i in geometry_df.index if not -126<geometry_df['southwest_lon'][i]<-110]

details_df = details_df.drop([i for i in details_df.index if details_df['place_id'][i] in bad_locations], axis = 0)
geometry_df = geometry_df.drop([i for i in geometry_df.index if geometry_df['place_id'][i] in bad_locations], axis=0)
reviews_df = reviews_df.drop([i for i in reviews_df.index if reviews_df['place_id'][i] in bad_locations], axis=0)
photos_df = photos_df.drop([i for i in photos_df.index if photos_df['place_id'][i] in bad_locations], axis=0)


In [151]:
c = Counter(details_df["types"][0])
for x in details_df["types"][1:]:
    c.update(x)
print(c)

Counter({'establishment': 718, 'point_of_interest': 717, 'food': 622, 'tourist_attraction': 212, 'bar': 121, 'store': 94, 'liquor_store': 76, 'restaurant': 28, 'lodging': 9, 'art_gallery': 7, 'park': 6, 'cafe': 4, 'night_club': 2, 'grocery_or_supermarket': 2, 'clothing_store': 2, 'bakery': 2, 'storage': 2, 'furniture_store': 2, 'home_goods_store': 2, 'route': 1, 'convenience_store': 1, 'gas_station': 1, 'local_government_office': 1, 'transit_station': 1, 'subpremise': 1, 'department_store': 1, 'hardware_store': 1, 'electronics_store': 1, 'natural_feature': 1, 'meal_takeaway': 1, 'rv_park': 1})


In [165]:
## investigate the following types: department_store, hardware_store, electronics_store, night_club, grocery_or_supermarket,
## clothing_store, bakery, storage, furniture_store, home_goods_store, route, convenience_store, gas_station, local_government_office, transit_station
types = ["department_store", "hardware_store", "electronics_store", "night_club", "grocery_or_supermarket", "clothing_store", "bakery", "storage", "furniture_store", "home_goods_store", "route", "convenience_store", "gas_station", "local_government_office", "transit_station"]
investigate = set()
for t in types:
    for i in details_df.index:
#         print(t, "   ||   ", details_df['types'][i])
        if t in details_df['types'][i]:
            investigate.add(i)
investigate = list(investigate)
investigate.sort()
details_df.loc[investigate, ["name", "website"]]

,name,website
30,Southwest Griffin Drive,NaN
32,Circle K,https://www.circlek.com/store-locator/US/vanco...
39,Pierce County Animal Control,https://www.co.pierce.wa.us/3966/Animal-Contro...
82,ENSO Winery,http://www.ensowinery.com/
134,Cyril's,NaN
151,SE 164th Ave & 12th St,NaN
365,Durant at Red Ridge Farms,http://durantoregon.com/
371,Westport Winery Garden Resort,http://www.westportwinery.com/
389,Arcane Cellars Winery,http://arcanecellars.com/
396,Community Warehouse (Estate Store),http://communitywarehouse.org/estatestore


In [168]:
# remove 30,32,39,134,151,396,484,565
bad_locations = set([details_df["place_id"][i] for i in [30,32,39,134,151,396,484,565]])

details_df = details_df.drop([i for i in details_df.index if details_df['place_id'][i] in bad_locations], axis = 0)
geometry_df = geometry_df.drop([i for i in geometry_df.index if geometry_df['place_id'][i] in bad_locations], axis=0)
reviews_df = reviews_df.drop([i for i in reviews_df.index if reviews_df['place_id'][i] in bad_locations], axis=0)
photos_df = photos_df.drop([i for i in photos_df.index if photos_df['place_id'][i] in bad_locations], axis=0)


KeyError: 30

In [169]:
c = Counter(details_df["name"][0].lower().split())
for x in details_df["name"][1:]:
    c.update(x.lower().split())
print(c)

Counter({'winery': 184, 'vineyards': 156, 'vineyard': 152, 'cellars': 82, '&': 59, 'wine': 58, 'tasting': 57, 'wines': 52, 'room': 48, 'estate': 34, 'hill': 22, 'and': 20, 'ridge': 19, 'family': 18, 'the': 17, 'valley': 13, 'creek': 12, 'bar': 10, 'domaine': 10, 'llc': 8, 'walla': 8, 'co.': 7, '-': 7, 'company': 7, 'hood': 6, 'de': 6, 'by': 5, 'of': 5, 'co': 5, 'farm': 5, 'willamette': 5, 'at': 5, 'hills': 5, 'mountain': 5, 'estates': 4, 'appointment': 4, 'cellar': 4, 'terra': 4, 'carlo': 4, 'carlton': 4, 'river': 4, 'seven': 4, 'oregon': 4, 'inc': 4, 'winery,': 4, 'red': 4, 'stone': 4, 'house': 4, 'david': 3, 'wineries': 3, 'don': 3, 'vineyard,': 3, 'chateau': 3, 'ranch': 3, 'view': 3, 'cathedral': 3, 'urban': 3, 'belle': 3, 'evoke': 3, 'bridge': 3, 'lachini': 3, '+': 3, 'laurel': 3, 'gorge': 3, 'cooper': 3, 'pheasant': 3, 'vintners': 2, 'lumos': 2, 'high': 2, 'road': 2, 'distillery': 2, 'three': 2, 'lake': 2, 'legacy': 2, 'owen': 2, 'roe': 2, 'garden': 2, 'white': 2, 'rose': 2, 'j.':

In [174]:
details_df.columns

Index(['address_components', 'adr_address', 'business_status',
       'current_opening_hours', 'formatted_address', 'formatted_phone_number',
       'icon', 'icon_background_color', 'icon_mask_base_uri',
       'international_phone_number', 'name', 'opening_hours', 'place_id',
       'plus_code', 'rating', 'reference', 'types', 'url',
       'user_ratings_total', 'utc_offset', 'vicinity', 'website',
       'permanently_closed', 'curbside_pickup', 'delivery', 'takeout',
       'dine_in', 'editorial_summary', 'price_level',
       'secondary_opening_hours'],
      dtype='object')

In [175]:
details_df["formatted_address"]

0          820 Mill Creek Rd, Walla Walla, WA 99362, USA
1           2444 Cobleigh Rd, Eagle Point, OR 97524, USA
2             8252 Redstone Ave SE, Salem, OR 97306, USA
3           245 N Front St, Central Point, OR 97502, USA
4                  140 NE Hill St, Albany, OR 97321, USA
                             ...                        
744    By Appointment Only, 15155 NE North Valley Rd,...
745              27415 Siuslaw Rd, Lorane, OR 97451, USA
747              272 Capital Ln, Roseburg, OR 97471, USA
748           9605 NE Fox Farm Rd, Dundee, OR 97115, USA
749              1242 SE Maple St, Dundee, OR 97115, USA
Name: formatted_address, Length: 712, dtype: object

In [188]:
details_df["street_address_1"] = details_df["formatted_address"].apply(lambda x: x.split(",")[:-3])
details_df["street_address_2"] = ""
details_df["city"] = details_df["formatted_address"].apply(lambda x: x.split(",")[-3])
details_df["province"] = details_df["formatted_address"].apply(lambda x: x.split(",")[-2].split()[0])
details_df["zipcode"] = details_df["formatted_address"].apply(lambda x: x.split(",")[-2].split()[1])
details_df["country_code"] = details_df["formatted_address"].apply(lambda x: x.split(",")[-1])
    
details_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 749
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   address_components          712 non-null    object
 1   adr_address                 712 non-null    object
 2   business_status             710 non-null    object
 3   current_opening_hours       486 non-null    object
 4   formatted_address           712 non-null    object
 5   formatted_phone_number      651 non-null    object
 6   icon                        712 non-null    object
 7   icon_background_color       712 non-null    object
 8   icon_mask_base_uri          712 non-null    object
 9   international_phone_number  651 non-null    object
 10  name                        712 non-null    object
 11  opening_hours               486 non-null    object
 12  place_id                    712 non-null    object
 13  plus_code                   709 non-null    object

In [201]:
# details_df['business_status'].unique()
bad_locations = set([details_df['place_id'][i] for i in details_df[details_df['business_status'] == 'CLOSED_PERMANENTLY'].index.to_list()])
[bad_locations.add(details_df['place_id'][i]) for i in details_df[details_df['permanently_closed']==True].index]

details_df = details_df.drop([i for i in details_df.index if details_df['place_id'][i] in bad_locations], axis = 0)
geometry_df = geometry_df.drop([i for i in geometry_df.index if geometry_df['place_id'][i] in bad_locations], axis=0)
reviews_df = reviews_df.drop([i for i in reviews_df.index if reviews_df['place_id'][i] in bad_locations], axis=0)
photos_df = photos_df.drop([i for i in photos_df.index if photos_df['place_id'][i] in bad_locations], axis=0)



In [206]:
## Rest the indexes. the gaps in their number order are becoming annoying
details_df = details_df.reset_index().drop('index', axis=1)
reviews_df = reviews_df.reset_index().drop('index', axis=1)
photos_df = photos_df.reset_index().drop('index',axis=1)
geometry_df = geometry_df.reset_index().drop('index',axis=1)

In [234]:
details_df = details_df.loc[:,['business_status','formatted_phone_number','name','place_id','rating','user_ratings_total','formatted_address','website',
'street_address_1', 'street_address_2', 'city', 'province', 'zipcode', 'country_code','url']]
details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   business_status         634 non-null    object
 1   formatted_phone_number  587 non-null    object
 2   name                    636 non-null    object
 3   place_id                636 non-null    object
 4   rating                  572 non-null    object
 5   user_ratings_total      572 non-null    object
 6   formatted_address       636 non-null    object
 7   website                 582 non-null    object
 8   street_address_1        636 non-null    object
 9   street_address_2        636 non-null    object
 10  city                    636 non-null    object
 11  province                636 non-null    object
 12  zipcode                 636 non-null    object
 13  country_code            636 non-null    object
 14  url                     636 non-null    object
dtypes: obj

In [243]:
## Convert the datatypes for the sql database
details_df['rating'] = pd.to_numeric(details_df['rating'])
details_df['user_ratings_total'] = pd.to_numeric(details_df['user_ratings_total'])
details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   business_status         634 non-null    object 
 1   formatted_phone_number  587 non-null    object 
 2   name                    636 non-null    object 
 3   place_id                636 non-null    object 
 4   rating                  572 non-null    float64
 5   user_ratings_total      572 non-null    float64
 6   formatted_address       636 non-null    object 
 7   website                 582 non-null    object 
 8   street_address_1        636 non-null    object 
 9   street_address_2        636 non-null    object 
 10  city                    636 non-null    object 
 11  province                636 non-null    object 
 12  zipcode                 636 non-null    object 
 13  country_code            636 non-null    object 
 14  url                     636 non-null    ob

In [216]:

reviews_df = reviews_df.loc[:, [col for col in reviews_df.columns if (col != 'profile_photo_url') and (col != 'language' )]]
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   author_name                572 non-null    object
 1   author_url                 572 non-null    object
 2   original_language          502 non-null    object
 3   rating                     572 non-null    object
 4   relative_time_description  572 non-null    object
 5   text                       572 non-null    object
 6   time                       572 non-null    object
 7   translated                 572 non-null    object
 8   place_id                   572 non-null    object
dtypes: object(9)
memory usage: 40.3+ KB


In [231]:
## Change the datatypes to the required sql data type
reviews_df['translated'] =  reviews_df['translated']==True
reviews_df['rating'] = pd.to_numeric(reviews_df["rating"])
reviews_df['time'] = pd.to_numeric(reviews_df["time"])
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   author_name                572 non-null    object
 1   author_url                 572 non-null    object
 2   original_language          502 non-null    object
 3   rating                     572 non-null    int64 
 4   relative_time_description  572 non-null    object
 5   text                       572 non-null    object
 6   time                       572 non-null    int64 
 7   translated                 572 non-null    bool  
 8   place_id                   572 non-null    object
dtypes: bool(1), int64(2), object(6)
memory usage: 36.4+ KB


In [244]:
photos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   photo_reference  4981 non-null   object
 1   place_id         4981 non-null   object
 2   height           4981 non-null   int64 
 3   width            4981 non-null   int64 
 4   uploader         4981 non-null   object
dtypes: int64(2), object(3)
memory usage: 194.7+ KB


In [251]:
photos_df['uploader'][0][0].exctract(r">(\s*\w+\s*\w*)\s*<")

AttributeError: 'str' object has no attribute 'exctract'

### Data Update
There are 74 Washington Vineyrd and wineries in the data set theat needs removed. when you add in the vineyard services and non-wine vineyards to the list, I am anticipating approximatly 20% of the data will dropped.     

sql notes: 
if_exists: "append"



In [170]:
reviews_df.to_csv("google_vineyard_reviews.csv", index=False)

In [171]:
details_df.to_csv("google_vineyard_details.csv", index=False)

In [172]:
geometry_df.to_csv("google_vineyard_maps_geometry.csv")

In [173]:
photos_df.to_csv("google_vineyard_photos.csv")

In [ ]:
from sqlalchemy import create_engine
db_address = config.db_type + config.db_user + config.db_pass + config.dbtype + config.db_aws
engine = create_engine(db_address)
inspector = inspect(engine)

In [149]:
reviews_df    

,author_name,author_url,language,original_language,profile_photo_url,rating,relative_time_description,text,time,translated,place_id
0,Smiledoc,https://www.google.com/maps/contrib/1063067683...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu_Gr...,5,4 years ago,"great wine, great people, great location and f...",1522099905,False,ChIJb21eb2cVolQRY6JGmq_XmwA
1,Kay Ross,https://www.google.com/maps/contrib/1123279401...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9H6...,5,a year ago,First time at Coria and not disappointed!\nCai...,1628467472,False,ChIJm-BEGj4GwFQRQz0MEeIFSLM
2,Joshua Smith,https://www.google.com/maps/contrib/1134752247...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu8AT...,5,2 years ago,Excellent service and solid wines. Small garde...,1599336320,False,ChIJ4zJjNBN8z1QRy_DnGt1EB-c
3,Linda Teppler (S0rceress0),https://www.google.com/maps/contrib/1049795854...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9Cs...,5,3 years ago,Just tried their Treos Brandy at the Corvallis...,1561834276,False,ChIJ6QmAe3sVwFQR7KVJoo1dw3k
4,Matias Sanchez,https://www.google.com/maps/contrib/1034946324...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9_K...,5,4 years ago,The wine here blows my mind away!,1533678636,False,ChIJPyUzT_0WlVQRPplhbRhoBv8
...,...,...,...,...,...,...,...,...,...,...,...
654,Danielle Verdugo,https://www.google.com/maps/contrib/1013689582...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu-zQ...,5,8 years ago,"A must see and taste!! Wonderful staff, beauti...",1407261261,False,ChIJNdln3D05wVQRP5h-7LTPjaI
656,Julie Helgason,https://www.google.com/maps/contrib/1159198501...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9fe...,5,3 years ago,Best Pinot Noirs in Douglas County! Also the P...,1569618105,False,ChIJX0B_1t4ZxFQRsVefqiKvc9I
657,Alaina Danae,https://www.google.com/maps/contrib/1045108330...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu8JQ...,5,3 years ago,This winery is absolutely amazing! The staff i...,1543869171,False,ChIJsfC-ZK9BlVQRdC0UjH76x58
658,Steve Yates,https://www.google.com/maps/contrib/1147946163...,NaN,NaN,https://lh3.googleusercontent.com/a/ALm5wu3BVu...,5,3 years ago,,1546546791,False,ChIJQaTQ6gxBlVQRGMITAAYqZ6U


## Data cleaning: 
place types that need investigated for not being a vineyard have the following types:
- storage
- rv_park
- cafe
- park
- art_gallery
- lodging
- store



Known Oregon Regions and Subregion AVA's

 - Willamette Valley
    - Chehalem Mountains
    - Dundee Hills
    - Yamhill-Carlton
    - Eola-Amity Hills
    - McMinnville
    - Laurelwood District
    - Ribbon Ridge
    - Van Duzer Corridor
    - Tualatin Hills
    - Mount Pisgah, Polk County
    - Lower Long Tom
 - Rouge Valley
    - Applegate Valley
 - Umpqua Valley
    - Elkton Oregon
    - Red Hill Douglas County
 - Columbia Gorge  
 - Columbia Valley
    - The Rocks District of Milton-Freewater
    - Walla Walla Valley
 - Snake River Valley    
 